In [1]:
import urllib, http.client
import time
import json
import hmac, hashlib

In [2]:
API_KEY ='K-3038939b9a19156293f232b73e6d2392c9799078'
API_SECRET = 'S-c25c5229f8be56020bcd495c8adac52e02ff0c4a'

In [4]:
CURRENCY_1 = 'BTC'
CURRENCY_2 = 'USD'
CURRENCY_1_MIN_QUANTITY = 0.001

In [5]:
ORDER_LIFE_TIME = 3 # минуты
STOCK_FEE = 0.002 # комиссия == 0.2%
AVG_PRICE_PERIOD = 90 # за какой период брать ср цену(минуты)
CAN_SPEND = 5 # сколько тратить CUR_2 при каждой покупке CUR_1
BLOCK_SPEND = 30  # ограничение CUR_2 на покупку
PROFIT_MARKUP = 0.001 # прибыль в 1%
DEBUG = True # инфа отладки
STOCK_TIME_OFFSET = 0 # если время биржи расходится с нашим

# обращение к api
API_URL = 'api.exmo.me'
API_VERSION = 'v1'
CURRENT_PAIR = CURRENCY_1 + '_' + CURRENCY_2

In [6]:
class ExmoAPI:
    def __init__(self, API_KEY, API_SECRET, API_URL = 'api.exmo.me', API_VERSION = 'v1'):
        self.API_URL = API_URL
        self.API_VERSION = API_VERSION
        self.API_KEY = API_KEY
        self.API_SECRET = bytes(API_SECRET, encoding='utf-8')        
    def sha512(self, data):
        H = hmac.new(key = self.API_SECRET, digestmod = hashlib.sha512)
        H.update(data.encode('utf-8'))
        return H.hexdigest()
    def api_query(self, api_method, params = {}):
        params['nonce'] = int(round(time.time() * 1000))
        params =  urllib.parse.urlencode(params)
        sign = self.sha512(params)
        headers = {
            "Content-type": "application/x-www-form-urlencoded",
            "Key": self.API_KEY,
            "Sign": sign
        }   
        conn = http.client.HTTPSConnection(self.API_URL, timeout=60)
        conn.request("POST", "/" + self.API_VERSION + "/" + api_method, params, headers)
        response = conn.getresponse().read()
        conn.close()
        
        try:
            obj = json.loads(response.decode('utf-8'))
            if 'error' in obj and obj['error']:
                print(obj['error'])
                raise sys.exit()
            return obj
        except json.decoder.JSONDecodeError:
            print('Error while parsing response:', response)
            raise sys.exit()

In [7]:
api = ExmoAPI(API_KEY, API_SECRET)
for i in api.api_query('user_info').items():
    print(i)

('uid', 1421646)
('server_date', 1530158845)
('balances', {'USD': '0', 'EUR': '0', 'RUB': '74', 'PLN': '0', 'UAH': '0', 'BTC': '0', 'LTC': '0', 'DOGE': '0', 'DASH': '0', 'ETH': '0', 'WAVES': '0', 'ZEC': '0', 'USDT': '0', 'XMR': '0', 'XRP': '0', 'KICK': '0', 'ETC': '0', 'BCH': '0', 'BTG': '0', 'EOS': '0', 'HBZ': '0', 'BTCZ': '0', 'DXT': '0'})
('reserved', {'USD': '0', 'EUR': '0', 'RUB': '20', 'PLN': '0', 'UAH': '0', 'BTC': '0', 'LTC': '0', 'DOGE': '0', 'DASH': '0', 'ETH': '0', 'WAVES': '0', 'ZEC': '0', 'USDT': '0', 'XMR': '0', 'XRP': '0', 'KICK': '0', 'ETC': '0', 'BCH': '0', 'BTG': '0', 'EOS': '0', 'HBZ': '0', 'BTCZ': '0', 'DXT': '0'})


# Get closed trades 

In [ ]:
'amount': '68.97540322',
'date': 'Jun|28|14:11:16|2018',
'price': '6102.30069999',
'quantity': '0.01130318',
'trade_id': 63463481,
'type': 'buy'},

In [11]:
large_pac = 10000
small_pac = 1000

In [342]:
def get_trades(l):
    trades = api.api_query('trades',
        {'pair':str(CURRENCY_1)+'_'+str(CURRENCY_2), 'limit':l}).items()
    trades_ = []
    for i in trades:
        trades_.append(i)
        
    trades = trades_[0][1]
    
    '''
    for i in range(0, len(trades)):
        t_d = time.ctime(trades[i].get('date')).split(' ')[1::]
        trades[i]['date'] = '|'.join(t_d)
    '''
    
    return trades[::-1] # traeds по времени 0 - старое, -1 - последнее новое

In [343]:
#метод объединения основного пакета с новым 
def unite_pacs():
    rotate_array = []
    for i in range(1, len(mid_pac)):
        if main_trades[-1].get('date') < mid_pac[-i].get('date'):
            rotate_array.append(mid_pac[-i]) 

    for a in rotate_array[::-1]:
        main_trades.append(a)

    del rotate_array

In [406]:
#получаем основной пакет 10к записей 
main_trades = get_trades(100000) 

In [415]:
#пакет, который обновит  основной при вызве его через интервал и вызове метода unite_pacs()
mid_pac = get_trades(50)

In [416]:
# последняя в главном списке  
print(time.ctime(main_trades[-1].get('date')))

#номер трейда в пакете, который обновляет -> будет добавлено 49 - num в основной пакет
print(mid_pac.index(main_trades[-1]))

Thu Jun 28 17:08:18 2018
4781


In [417]:
# добавляем обновленные трейды
unite_pacs()

In [418]:
len(main_trades)

10242